# 第5章: 大規模言語モデル

## 40. Zero-Shot推論

In [1]:
# Geminiのライブラリのインストール
!pip install -q -U google-generativeai

In [2]:
# APIキーの読み込み
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [29]:
import csv

model = genai.GenerativeModel("gemini-1.5-flash")
prompt = ""

# CSVファイルを読み込む
with open("./data/llm-prompts.csv", "r", encoding="utf-8") as f:
  reader = csv.DictReader(f)
  for row in reader:
    if int(row["id"]) == 401:
      prompt = row["question"]
      break

response = model.generate_content(prompt)
print(response.text)

# Gemini_APIの使用  https://qiita.com/zukki2/items/10bfeb1c4330aa18ff87

これらの出来事を年代順に並べると以下のようになります。

1. **イ　嵯峨天皇は，藤原冬嗣らを蔵人頭に任命した。**　嵯峨天皇の治世は809年から823年。藤原冬嗣は嵯峨天皇の治世中に蔵人頭を務めました。

2. **ウ　藤原良房は，承和の変後，藤原氏の中での北家の優位を確立した。**　承和の変は842年。この変の後、藤原良房は権勢を確固たるものにしました。

3. **ア　藤原時平は，策謀を用いて菅原道真を政界から追放した。**　菅原道真の左遷は894年。これはイ、ウの出来事の後です。


よって、年代順は イ → ウ → ア となります。



## 41. Few-Shot推論

In [53]:
model = genai.GenerativeModel("gemini-1.5-flash")
few_shot = ""
question = ""

# CSVファイルを読み込む
with open("./data/llm-prompts.csv", "r", encoding="utf-8") as f:
  reader = csv.DictReader(f)
  for row in reader:
    if (int(row["id"]) >= 410 and int(row["id"]) <= 414):
      few_shot += ("question:" + row["question"] + " answer:" + row["answer"] + " ")  # 質問と答えを連結
    elif (int(row["id"]) == 401):
      question = ("question:" + row["question"] + " answer:")
prompt = few_shot + question

response = model.generate_content(prompt)
print(response.text)

解答: イ→ウ→ア



## 42. 多肢選択問題の正解率

In [72]:
import time

model = genai.GenerativeModel("gemini-1.5-flash")
correct = []
answer = []

# CSVファイルを読み込む
# business_ethics.csvファイルのカラムをquestion,A,B,C,D,answerとする
with open("./data/business_ethics.csv", "r", encoding="utf-8-sig") as f:
  reader = csv.DictReader(f)
  for row in reader:
    prompt = ("question:" + row["question"] + "A:" + row["A"] + "B:" + row["B"] + "C:" + row["C"] + "D:" + row["D"] + "A, B, C, Dのいずれかの記号のみを出力してください。")
    response = model.generate_content(prompt)
    correct.append(row["answer"].strip())
    answer.append(response.text.strip())
    time.sleep(4) # 1分に15回のリクエスト制限

acc = len(set(correct) and set(answer)) / len(correct)
print(f"{acc * 100:.2f}%")

# sleep関数 https://qiita.com/shi_ei/items/2a88ba1796a4f8f7ad0a

5.81%


## 43. 応答のバイアス

In [73]:
model = genai.GenerativeModel("gemini-1.5-flash")
correct = []
answer = []

# CSVファイルを読み込む
with open("./data/business_ethics.csv", "r", encoding="utf-8-sig") as f:
  reader = csv.DictReader(f)
  for row in reader:
    # プロンプトの一部をエストニア語（母語話者が少ない）に変更
    prompt = ("küsimus:" + row["question"] + "A:" + row["A"] + "B:" + row["B"] + "C:" + row["C"] + "D:" + row["D"] + "Palun väljasta ainult üks sümbol A, B, C või D.")
    response = model.generate_content(prompt)
    correct.append(row["answer"].strip())
    answer.append(response.text.strip())
    time.sleep(4) # 1分に15回のリクエスト制限

acc = len(set(correct) and set(answer)) / len(correct)
print(f"{acc * 100:.2f}%")

5.81%


## 44. 対話

In [74]:
model = genai.GenerativeModel("gemini-1.5-flash")
question = ("つばめちゃんは渋谷駅から東急東横線に乗り、自由が丘駅で乗り換えました。東急大井町線の大井町方面の電車に乗り換えたとき、各駅停車に乗車すべきところ、間違えて急行に乗車してしまったことに気付きました。自由が丘の次の急行停車駅で降車し、反対方向の電車で一駅戻った駅がつばめちゃんの目的地でした。目的地の駅の名前を答えてください。")
response = model.generate_content(question)
print(response.text)

つばめちゃんが間違えて急行に乗車し、自由が丘の次の急行停車駅で降車したということは、次の駅は**武蔵小山**です。そこから反対方向の電車で一駅戻ると、目的地の駅は**戸越公園**になります。

